In [1]:

def custom_pFBA(model):
    sol = model.optimize()
    for rxn in model.reactions:
        if rxn.objective_coefficient != 0:
            rxn.lower_buond = round(sol.fluxes[rxn.id],3)
            rxn.upper_bound = round(sol.fluxes[rxn.id],3)
    from sweetlovegroup import FBA
    Irrev_model = FBA.rev2irrev(model)
    for rxn in Irrev_model.reactions:
        if rxn.upper_bound > 0:
            rxn.objective_coefficient = -1
        else:
            rxn.objective_coefficient = 1
    sol2 = Irrev_model.optimize()
    rxnSet=set()
    fluxDict = dict()
    for rxn in Irrev_model.reactions.query("_reverse"):
        rxnSet.add(rxn.id)
        rxnSet.add(rxn.id.replace("_reverse",""))
        fluxDict[rxn.id.replace("_reverse","")]=sol2.fluxes[rxn.id]+sol2.fluxes[rxn.id.replace("_reverse","")]
    for rxn in Irrev_model.reactions:
        if rxn.id in rxnSet:
            continue
        else:
            fluxDict[rxn.id]=sol2.fluxes[rxn.id]
    sol3 = sol2
    sol3.fluxes = fluxDict
    return sol3


def remove_metabolite_from_reaction(rxn,mets):
    '''
    This functions removes a list of metabolites from a reaction
    '''
    for met in mets:
        if met in rxn.metabolites.keys():
            coeff = rxn.metabolites.get(met)
            rxn.add_metabolites({met:-1*coeff})
        else:
            print("Metabolite "+met.id+" not present in reaction "+rxn.id)
    return rxn

import pandas as pd


from cobra import io,flux_analysis
from cobra.core import Reaction, Metabolite

#import model. Update file name and location in the next line
cobra_model = io.sbml.read_sbml_model("./../Data/PlantCoreMetabolism_v2_0_0.xml")


#Remove all metabolites except sucrose from Phloem
rxn = cobra_model.reactions.get_by_id("Phloem_output_tx")
mets2remove = list()


JATPase = 0

JNADPHox = 0

#no external sucrose or glucose
cobra_model.reactions.get_by_id("Sucrose_tx").lower_bound = 0
cobra_model.reactions.get_by_id("Sucrose_tx").upper_bound = 0
cobra_model.reactions.get_by_id("GLC_tx").lower_bound = 0
cobra_model.reactions.get_by_id("GLC_tx").upper_bound = 0

#no external light energy
cobra_model.reactions.get_by_id("Photon_tx").lower_bound = 0
cobra_model.reactions.get_by_id("Photon_tx").upper_bound = 0

#set export of sugars as objective
cobra_model.reactions.get_by_id("Phloem_output_tx").objective_coefficient=1

#add source reaction for TP
rxn = Reaction("GAP_tx",name = "TP source")
rxn.add_metabolites({cobra_model.metabolites.get_by_id("GAP_c"):1})
rxn.upper_bound = 1000
rxn.lower_bound = 0
cobra_model.add_reaction(rxn)


rxn = Reaction("G3P_tx",name = "PGA source")
rxn.add_metabolites({cobra_model.metabolites.get_by_id("G3P_c"):1})
rxn.upper_bound = 1000
rxn.lower_bound = 0
cobra_model.add_reaction(rxn)

#add source reaction for TP
rxn = Reaction("GLYCOLATE_tx",name = "Glycolate source")
rxn.add_metabolites({cobra_model.metabolites.get_by_id("GLYCOLLATE_c"):1})
rxn.upper_bound = 1000
rxn.lower_bound = 0
cobra_model.add_reaction(rxn)

#add source reaction for TP
rxn = Reaction("GLYCERATE_tx",name = "Glycerate sink")
rxn.add_metabolites({cobra_model.metabolites.get_by_id("GLYCERATE_c"):-1})
rxn.upper_bound = 1000
rxn.lower_bound = 0
cobra_model.add_reaction(rxn)

#remove mGS and cGS
cobra_model.reactions.get_by_id("GLUTAMINESYN_RXN_m").lower_bound = 0
cobra_model.reactions.get_by_id("GLUTAMINESYN_RXN_m").upper_bound = 0
cobra_model.reactions.get_by_id("GLUTAMINESYN_RXN_c").lower_bound = 0
cobra_model.reactions.get_by_id("GLUTAMINESYN_RXN_c").upper_bound = 0

#remove glutamine synthetase and glutamate synthase
#rxn = cobra_model.reactions.get_by_id("GLUTAMATE_SYNTHASE_FERREDOXIN_RXN_p")
#mets2remove=[cobra_model.metabolites.get_by_id("Reduced_ferredoxins_p"),cobra_model.metabolites.get_by_id("Oxidized_ferredoxins_p")]
#remove_metabolite_from_reaction(rxn,mets2remove)
#rxn = cobra_model.reactions.get_by_id("GLUTAMINESYN_RXN_p")
#mets2remove=[cobra_model.metabolites.get_by_id("ATP_p"),cobra_model.metabolites.get_by_id("aATP_p")]
#remove_metabolite_from_reaction(rxn,mets2remove)

#rxn = Reaction("NrefixationCostbypass")
#rxn.add_metabolites({cobra_model.metabolites.get_by_id("GLT_x"):1,cobra_model.metabolites.get_by_id("2_KETOGLUTARATE_x"):-1,cobra_model.metabolites.get_by_id("AMMONIUM_m"):-1})
#rxn.lower_bound = 0
#rxn.upper_bound = 1000
#cobra_model.add_reaction(rxn)

#provide energy for N fixation
#rxn = Reaction("NrefixationEnergy")
#rxn.add_metabolites({cobra_model.metabolites.get_by_id("ATP_p"):0.9,cobra_model.metabolites.get_by_id("aATP_p"):0.1,cobra_model.metabolites.get_by_id("ADP_p"):-0.8,cobra_model.metabolites.get_by_id("aADP_p"):-0.2,cobra_model.metabolites.get_by_id("Pi_p"):-1,cobra_model.metabolites.get_by_id("Reduced_ferredoxins_p"):2,cobra_model.metabolites.get_by_id("Oxidized_ferredoxins_p"):-2})
#rxn.lower_bound = 0
#rxn.upper_bound = 1000
#met = Metabolite("NrefixEnergyConstraint")
#rxn.add_metabolites({met:1})
#cobra_model.add_reaction(rxn)
#cobra_model.reactions.get_by_id("GCVMULTI_RXN_m").add_metabolites({met:-1})

#turn off phosphoserine transaminase
#cobra_model.reactions.get_by_id("PSERTRANSAM_RXN_p").lower_bound = 0
#cobra_model.reactions.get_by_id("PSERTRANSAM_RXN_p").upper_bound = 0


cobra_model.reactions.get_by_id("Pi_ec").lower_bound = -1000
cobra_model.reactions.get_by_id("Pi_ec").upper_bound = 1000

#add malate and citrate accumulation reactions
rxn = Reaction("MAL_v_accumulation")
rxn.add_metabolites({cobra_model.metabolites.MAL_v:-0.7,cobra_model.metabolites.aMAL_v:-0.3})
rxn.lower_bound = -1000
rxn.upper_bound = 1000
cobra_model.add_reaction(rxn)
rxn = Reaction("CIT_v_accumulation")
rxn.add_metabolites({cobra_model.metabolites.CIT_v:-0.5,cobra_model.metabolites.aCIT_v:-0.5})
rxn.lower_bound = -1000
rxn.upper_bound = 1000
cobra_model.add_reaction(rxn)


temp = cobra_model.copy()

PPFD = 500
#constrain maintenace
ATPase = (0.0049*PPFD) + 2.7851
ATPase = round(ATPase,2)
temp.reactions.get_by_id("ATPase_tx").lower_bound = ATPase
temp.reactions.get_by_id("ATPase_tx").upper_bound = ATPase
temp.reactions.get_by_id("NADPHoxc_tx").lower_bound = ATPase/9
temp.reactions.get_by_id("NADPHoxc_tx").upper_bound = ATPase/9
temp.reactions.get_by_id("NADPHoxp_tx").lower_bound = ATPase/9
temp.reactions.get_by_id("NADPHoxp_tx").upper_bound = ATPase/9
temp.reactions.get_by_id("NADPHoxm_tx").lower_bound = ATPase/9
temp.reactions.get_by_id("NADPHoxm_tx").upper_bound = ATPase/9

#constraint TP flux
temp.reactions.get_by_id("GAP_tx").lower_bound = 1.884570e+01
temp.reactions.get_by_id("GAP_tx").upper_bound = 1.884570e+01
temp.reactions.get_by_id("G3P_tx").lower_bound = 1.884570e+01
temp.reactions.get_by_id("G3P_tx").upper_bound = 1.884570e+01

#constraint glycollate and glycerate fluxes flux
temp.reactions.get_by_id("GLYCOLATE_tx").lower_bound = 7.782878e+00
temp.reactions.get_by_id("GLYCOLATE_tx").upper_bound = 7.782878e+00
temp.reactions.get_by_id("GLYCERATE_tx").lower_bound = 3.891439e+00
temp.reactions.get_by_id("GLYCERATE_tx").upper_bound = 3.891439e+00

#temp.reactions.get_by_id("NrefixationCostbypass").lower_bound = df270["Vt_glycolate"][i]
#temp.reactions.get_by_id("NrefixationCostbypass").upper_bound = df270["Vt_glycolate"][i]
#temp.reactions.get_by_id("NrefixationEnergy").lower_bound = df270["Vt_glycerate"][i]
#temp.reactions.get_by_id("NrefixationEnergy").upper_bound = df270["Vt_glycerate"][i]

temp.reactions.get_by_id("MAL_v_accumulation").lower_bound = 0.0698903487288*(4.630002e-01-1.418010e-02)
temp.reactions.get_by_id("MAL_v_accumulation").upper_bound = 0.0698903487288*(4.630002e-01-1.418010e-02)

temp.reactions.get_by_id("CIT_v_accumulation").lower_bound = -0.056884259879*(4.630002e-01-1.418010e-02)
temp.reactions.get_by_id("CIT_v_accumulation").upper_bound = -0.056884259879*(4.630002e-01-1.418010e-02)



for rxn in cobra_model.reactions:
    if rxn.lower_bound == -1000:
        rxn.lower_boudn = -3000
    if rxn.upper_bound == 1000:
        rxn.upper_bound = 3000


#ADD ATP source reaction in FBA to represent ATP from JATPase
rxn = Reaction("ATP_source_from_ODE")
rxn.add_metabolites({temp.metabolites.get_by_id("ADP_p"):-0.8,
                     temp.metabolites.get_by_id("aADP_p"):-0.2,
                     temp.metabolites.get_by_id("Pi_p"):-1,
                     temp.metabolites.get_by_id("PROTON_p"):-0.9,
                     temp.metabolites.get_by_id("ATP_p"):0.9,
                     temp.metabolites.get_by_id("aATP_p"):0.1,
                     temp.metabolites.get_by_id("WATER_p"):1})
rxn.lower_bound = JATPase
rxn.upper_bound = JATPase
temp.add_reaction(rxn)

#ADD NADPH source reaction in FBA to represent NADPH from ODE
rxn = Reaction("NADPH_source_from_ODE")
rxn.add_metabolites({temp.metabolites.get_by_id("NADP_p"):-1,
                     temp.metabolites.get_by_id("WATER_p"):-1,
                     temp.metabolites.get_by_id("NADPH_p"):1,
                     temp.metabolites.get_by_id("OXYGEN_MOLECULE_p"):1,
                     temp.metabolites.get_by_id("PROTON_p"):1})
rxn.lower_bound = JNADPHox
rxn.upper_bound = JNADPHox
temp.add_reaction(rxn)

#check if model works
temp.solver="glpk"
#sol = custom_pFBA(temp)
try:
    sol = flux_analysis.parsimonious.optimize_minimal_flux(temp)
except:
    sol = custom_pFBA(temp)
rxn =  temp.reactions.get_by_id("Phloem_output_tx")
met = temp.metabolites.sSUCROSE_b
print("Sucrose export rate ="+str(rxn.metabolites[met]*sol.fluxes[rxn.id]))

total = JATPase
for rxn in temp.metabolites.ATP_p.reactions:
    if round(rxn.flux,3) != 0:
        coeff1 = rxn.metabolites[temp.metabolites.ATP_p]
        coeff2 = rxn.metabolites[temp.metabolites.aATP_p]
        ATPflux = sol.fluxes[rxn.id]*(coeff1+coeff2)
        #print(rxn.id+"\t"+str(ATPflux)+"="+str(total))
        if rxn.id == "ATP_ADP_Pi_pc":
            total = total + ATPflux
            print(ATPflux)
print("Extra APTase flux ="+str(total))


total = JNADPHox
for rxn in temp.metabolites.NADPH_p.reactions:
    if round(rxn.flux,3) != 0:
        coeff1 = rxn.metabolites[temp.metabolites.NADPH_p]
        NADPHflux = sol.fluxes[rxn.id]*(coeff1)
        #print(rxn.id+"\t"+str(NADPHflux)+"="+str(total))
        if rxn.id == "MALATE_DEH_RXN_p":
            total = total + NADPHflux
            print(NADPHflux)
for rxn in temp.metabolites.NADH_p.reactions:
    if round(rxn.flux,3) != 0:
        coeff1 = rxn.metabolites[temp.metabolites.NADH_p]
        NADPHflux = sol.fluxes[rxn.id]*(coeff1)
        #print(rxn.id+"\t"+str(NADPHflux)+"="+str(total))
        if rxn.id == "MALATE_DEH_RXN_p":
            total = total + NADPHflux
            print(NADPHflux)
print("Extra NADPH flux ="+str(total))


C:\ProgramData\Anaconda3\lib\site-packages\pandas\compat\_optional.py:106: UserWarning: Pandas requires version '1.2.1' or newer of 'bottleneck' (version '0+unknown' currently installed).
  warnings.warn(msg, UserWarning)


Academic license - for non-commercial use only - expires 2021-06-05
Using license file C:\Users\sanus\gurobi.lic
Read LP format model from file C:\Users\sanus\AppData\Local\Temp\tmpctv2yh6b.lp
Reading time = 0.02 seconds
: 861 rows, 1796 columns, 8940 nonzeros
Sucrose export rate =-7.025139382476676
5.111860307986952
Extra APTase flux =5.111860307986952
4.696590496309672
Extra NADPH flux =4.696590496309672


In [6]:
reaction_list = [temp.reactions.MALATE_DEH_RXN_p,temp.reactions.ATP_ADP_Pi_pc]

In [10]:
from cobra.flux_analysis import flux_variability_analysis
fva_result_pFBA = flux_variability_analysis(temp,reaction_list,pfba_factor=1)

In [9]:
fva_result

,minimum,maximum
MALATE_DEH_RXN_p,-982.962169,999.995986
ATP_ADP_Pi_pc,-1000.000000,988.163934


In [11]:
fva_result_pFBA

,minimum,maximum
MALATE_DEH_RXN_p,4.69659,4.696590
ATP_ADP_Pi_pc,-5.11186,-0.266793


In [23]:
for rxn in met.reactions:
    print(rxn.reaction)

0.5 ADP_c + 0.9 ATP_p + 0.1 PROTON_p + 0.7 Pi_c + 0.5 aADP_c + 0.1 aATP_p + 0.3 aPi_c + hypo1 <=> 0.8 ADP_p + 0.65 ATP_c + 0.45 PROTON_c + Pi_p + 0.2 aADP_p + 0.35 aATP_c
PYRUVATE_p + hypo1 <=> PYRUVATE_c
 --> hypo1


In [36]:
temp3 = temp.copy()

from cobra.core import Metabolite, Reaction
met = Metabolite("hypo1")

temp3.reactions.ATP_ADP_Pi_pc.add_metabolites({met:-1})
temp3.reactions.PYRUVATE_pc.add_metabolites({met:1})

rxn = Reaction("hypo2")
rxn.add_metabolites({met:-1})
rxn.lower_bound = 0
rxn.upper_bound = 1000
temp3.add_reactions([rxn,])

sol = flux_analysis.parsimonious.optimize_minimal_flux(temp3)
reaction_list = [temp3.reactions.MALATE_DEH_RXN_p,
                 temp3.reactions.ATP_ADP_Pi_pc,
                 temp3.reactions.get_by_id("hypo2"),
                 temp3.reactions.PYRUVATE_pc,
                 temp3.reactions.PEP_Pi_pc]
fva_result_pFBA2 = flux_variability_analysis(temp3,reaction_list,pfba_factor=1)

In [37]:
fva_result_pFBA2

,minimum,maximum
MALATE_DEH_RXN_p,4.696590e+00,4.696590
ATP_ADP_Pi_pc,-5.111860e+00,-0.266793
hypo2,5.111860e+00,5.111860
PYRUVATE_pc,1.848668e-12,4.845068
PEP_Pi_pc,-5.116851e+00,-0.271783


In [32]:
temp3 = temp.copy()

from cobra.core import Metabolite, Reaction
met1 = Metabolite("hypo1")
met2 = Metabolite("hypo2")
met3 = Metabolite("hypo3")

temp3.reactions.ATP_ADP_Pi_pc.add_metabolites({met1:-1})
temp3.reactions.PEP_Pi_pc.add_metabolites({met2:-1})
temp3.reactions.PYRUVATE_pc.add_metabolites({met3:1})

rxn1 = Reaction("hypo1")
rxn1.add_metabolites({met1:-1})
rxn1.lower_bound = 0
rxn1.upper_bound = 1000

rxn2 = Reaction("hypo2")
rxn2.add_metabolites({met2:-1})
rxn2.lower_bound = -1000
rxn2.upper_bound = 1000

rxn3 = Reaction("hypo3")
rxn3.add_metabolites({met3:-1})
rxn3.lower_bound = -1000
rxn3.upper_bound = 1000

rxn4 = Reaction("hypo4")
rxn4.add_metabolites({met1:1,met2:-1,met3:-1})
rxn4.lower_bound = 0
rxn4.upper_bound = 1000

temp3.add_reactions([rxn1,rxn2,rxn3,rxn4])

sol = flux_analysis.parsimonious.optimize_minimal_flux(temp3)
reaction_list = [temp3.reactions.MALATE_DEH_RXN_p,
                 temp3.reactions.ATP_ADP_Pi_pc,
                 temp3.reactions.get_by_id("hypo1"),
                 temp3.reactions.get_by_id("hypo2"),
                 temp3.reactions.get_by_id("hypo3"),
                 temp3.reactions.get_by_id("hypo4"),
                 temp3.reactions.PYRUVATE_pc,
                 temp3.reactions.PEP_Pi_pc]
fva_result_pFBA = flux_variability_analysis(temp3,reaction_list,pfba_factor=1)

In [33]:
fva_result_pFBA

,minimum,maximum
MALATE_DEH_RXN_p,4.696590e+00,4.696590e+00
ATP_ADP_Pi_pc,-5.111860e+00,-5.111860e+00
hypo1,5.111860e+00,5.111860e+00
hypo2,-1.479461e-12,2.717834e-01
hypo3,-2.219191e-12,4.438382e-12
hypo4,0.000000e+00,4.438382e-12
PYRUVATE_pc,-1.479461e-12,4.438382e-12
PEP_Pi_pc,-2.717834e-01,1.109596e-12


In [8]:
def constrainSumOfFluxes(cobra_model, rxn2avoid,SFvalue,objvalue,weightings):
    from cobra.core import Metabolite, Reaction
    
    temp=cobra_model.copy()
    SFMet = Metabolite("SFMet",name="Sum of fluxes pseudometabolite",compartment="c2")
    for rxn in cobra_model.reactions:
        if not rxn2avoid.__contains__(rxn.id):
            if rxn.id.__contains__("reverse"):
                temp.reactions.get_by_id(rxn.id).add_metabolites({SFMet:-1*weightings[rxn.id.replace("_reverse","")]})
            else:
                temp.reactions.get_by_id(rxn.id).add_metabolites({SFMet:1*weightings[rxn.id.replace("_reverse","")]})
    SFRxn = Reaction("SFRxn",name="Sum of fluxes pseudoreaction")
    SFRxn.add_metabolites({SFMet:-1})
    SFRxn.lower_bound=SFvalue
    SFRxn.upper_bound=SFvalue
    temp.add_reaction(SFRxn)
    if (not objvalue=="") and (len([rxn for rxn in temp.reactions if rxn.objective_coefficient==1]) == 1):
        for rxn in [rxn for rxn in temp.reactions if rxn.objective_coefficient==1]:
            rxn.lower_bound=float(objvalue)
            rxn.upper_bound=float(objvalue)
    return temp


#Function to convert any model with reversible reactions to a copy of the same m-
#-odel with only irreversible reactions. ID of reverse reactions are generated by
#suffixing "_reverse" to the ID of the orignal reaction.
#args: 1) a cobra model
#output: a cobra model with only irreversible reactions
def rev2irrev(cobra_model):
  exp_model=cobra_model.copy()

  for RXN in cobra_model.reactions:
    rxn=exp_model.reactions.get_by_id(RXN.id)
    if (rxn.lower_bound < 0):
      rxn_reverse = rxn.copy()
      rxn_reverse.id = "%s_reverse" %(rxn.id)
      rxn.lower_bound = 0
      rxn_reverse.upper_bound = 0
      exp_model.add_reaction(rxn_reverse)

  return exp_model




#Function to constraint sum of fluxes when performing FBA
#args: 1) a cobra model, 2) a python list of reactions to leave out from constrai-
#-nt, 3) the float value that sum of fluxes must be constrained to & 4) value obj-
#-ective function needs to be constraint to (provide "" to avoid constraining obj-
#ective function) 5) Flux weightings
#output: a cobra model with sum of fluxes constrained to
def constrainSumOfFluxes(cobra_model, rxn2avoid,SFvalue,objvalue,weightings):
  from cobra.core import Metabolite, Reaction

  temp=cobra_model.copy()
  SFMet = Metabolite("SFMet",name="Sum of fluxes pseudometabolite",compartment="c2")
  for rxn in cobra_model.reactions:
	  if not rxn2avoid.__contains__(rxn.id):
		  if rxn.id.__contains__("reverse"):
			  temp.reactions.get_by_id(rxn.id).add_metabolites({SFMet:-1*weightings[rxn.id.replace("_reverse","")]})
		  else:
			  temp.reactions.get_by_id(rxn.id).add_metabolites({SFMet:1*weightings[rxn.id.replace("_reverse","")]})
  SFRxn = Reaction("SFRxn",name="Sum of fluxes pseudoreaction")
  SFRxn.add_metabolites({SFMet:-1})
  SFRxn.lower_bound=SFvalue
  SFRxn.upper_bound=SFvalue
  temp.add_reaction(SFRxn)
  if (not objvalue=="") and (len([rxn for rxn in temp.reactions if rxn.objective_coefficient==1]) == 1):
    for rxn in [rxn for rxn in temp.reactions if rxn.objective_coefficient==1]:
      rxn.lower_bound=float(objvalue)
      rxn.upper_bound=float(objvalue)
  return temp



#################################################################################
# This function is a modified version of cobrapy add_pfba function			#
#										#
#################################################################################

def add_pfba_Weighted(model, weightings, objective=None, fraction_of_optimum=1.0):
    """Add pFBA objective
    Add objective to minimize the summed flux of all reactions to the
    current objective.
    See Also
    -------
    pfba
    Parameters
    ----------
    model : cobra.Model
        The model to add the objective to
    objective :
        An objective to set in combination with the pFBA objective.
    fraction_of_optimum : float
        Fraction of optimum which must be maintained. The original objective
        reaction is constrained to be greater than maximal_value *
        fraction_of_optimum.
    """
    if objective is not None:
        model.objective = objective
    if model.solver.objective.name == '_pfba_objective':
        raise ValueError('The model already has a pFBA objective.')
    sutil.fix_objective_as_constraint(model, fraction=fraction_of_optimum)
    reaction_variables = ((rxn.forward_variable, rxn.reverse_variable)
                          for rxn in model.reactions)
    variables = chain(*reaction_variables)
    model.objective = model.problem.Objective(
        Zero, direction='min', sloppy=True, name="_pfba_objective")
    #print([v for v in variables])
    tempDict = dict()
    for v in variables:
        w = str(v).split("=")[1].replace(" ","").replace("<","")
        found=False
        for rxn in weightings.keys():
            if w.__contains__(rxn):
                #print(v)
                #print(rxn)
                tempDict[v]=weightings[rxn]
                found=True
                break
        if not found:
            #print("Weightings for reaction "+w+" not found, so assuming weighting = 1")
            tempDict[v] = 1
    model.objective.set_linear_coefficients(tempDict)

#Function to perform FVA analysis which maintains sum of fluxes at a minimal val-
#ue
#args: 1) a cobra model 2) Objective 3) reaction to avoid when constraining sum
#of fluxes 4) reaction list for FVA 5) solver used to perform FVA
#output: a cobra model with FVA as an attribute called fva
def FBA_FVA_run(cobra_model,obj,rxn2avoid = [],rxnlist=[],solver="",weightings={}):
  #from sweetlovegroup.transform import rev2irrev
  #from sweetlovegroup.constraints import constrainSumOfFluxes
  #from sweetlovegroup.FBA import pfba_Weighted
  from cobra import flux_analysis

  if len(weightings)>0:
    weightings_submittied=True
  else:
    weightings_submittied=False

  if len(rxnlist)==0:
    rxnlist = cobra_model.reactions
  for rxn in cobra_model.reactions:
    if not rxn.id in weightings.keys():
      weightings[rxn.id]=1
      if weightings_submittied:
          print("Warning")
          print(rxn.id)
          return
  print("Runing pFBA")
  solution = pfba_Weighted(cobra_model,weightings)
  objvalue = solution.fluxes.get(obj.id)
  a = 0
  for i in cobra_model.reactions:
    a = a + abs(solution.fluxes.get(i.id)*weightings[i.id])

  sumOfFluxes = a

  cobra_model2 = cobra_model.copy()
  irr_model = rev2irrev(cobra_model2)
  print("Setting SOF model")
  sfmodel = constrainSumOfFluxes(irr_model,rxn2avoid,sumOfFluxes,objvalue,weightings)
  rxnlist2 = list()
  if len(rxnlist) == len(cobra_model.reactions):
    rxnlist2 = sfmodel.reactions
  else:
    for rxn in rxnlist:
      if rxn.lower_bound<0 and rxn.upper_bound>0:
        rxnlist2.append(sfmodel.reactions.get_by_id(rxn.id+"_reverse"))
      rxnlist2.append(sfmodel.reactions.get_by_id(rxn.id))
  #print("Rxn list ="+str(rxnlist2))
  print("Running FVA")

  if solver != "":
    import optlang
    if optlang.available_solvers.keys().__contains__(solver) and optlang.available_solvers[solver]:
      sfmodel.solver=solver
    else:
      print("Requested solver "+solver+" not available, using current model solver...")
  fva = flux_analysis.flux_variability_analysis(sfmodel,reaction_list = rxnlist2)
  print("Processing results")

  fva2=dict()
  for mode in fva.keys():
    if mode == "maximum":
      tempdict = dict()
      FVArxnSet = set()
      for rxn in fva[mode].keys():
        if rxn.__contains__("_reverse"):
          rxn = rxn.replace("_reverse","")
        if FVArxnSet.__contains__(rxn):
          continue
        FVArxnSet.add(rxn)
        if not fva[mode].keys().__contains__(rxn+"_reverse"):
          maxi = fva[mode][rxn]
        else:
          maxi = fva[mode][rxn]+fva[mode][rxn+"_reverse"]
        tempdict[rxn]=maxi
    else:
      tempdict=dict()
      FVArxnSet = set()
      for rxn in fva[mode].keys():
        if rxn.__contains__("_reverse"):
          rxn = rxn.replace("_reverse","")
        if FVArxnSet.__contains__(rxn):
          continue
        FVArxnSet.add(rxn)
        if not fva[mode].keys().__contains__(rxn+"_reverse"):
          mini = fva[mode][rxn]
        else:
          mini = fva[mode][rxn]+fva[mode][rxn+"_reverse"]
        tempdict[rxn]=mini
    fva2[mode]=tempdict

  sfmodel.fva = fva
  cobra_model.fva = fva2
  cobra_model.solution = solution
  return cobra_model


In [12]:
temp4 = temp.copy()

from cobra.core import Metabolite, Reaction

weightings = dict()
for rxn in temp4.reactions:
    if rxn.id=="ATP_ADP_Pi_pc":
        weightings[rxn.id]=0.5
    else:
        weightings[rxn.id]=1


sol = pfba_Weighted(temp4,weightings)
reaction_list = [temp4.reactions.MALATE_DEH_RXN_p,
                 temp4.reactions.ATP_ADP_Pi_pc,
                 temp4.reactions.PYRUVATE_pc,
                 temp4.reactions.PEP_Pi_pc]
fva_result_pFBA3 = FBA_FVA_run(temp4,temp4.reactions.get_by_id("Phloem_output_tx"),
                                   rxnlist=reaction_list,weightings=weightings)

Runing pFBA
Setting SOF model
Running FVA
Processing results


In [13]:
fva_result_pFBA3.fva

{'minimum': {'MALATE_DEH_RXN_p': 4.696590496296695,
  'ATP_ADP_Pi_pc': -5.1118603079811855,
  'PYRUVATE_pc': -1.6200439457361936e-11,
  'PEP_Pi_pc': -0.2717833824432152},
 'maximum': {'MALATE_DEH_RXN_p': 4.696590496314159,
  'ATP_ADP_Pi_pc': -5.111860307912636,
  'PYRUVATE_pc': 4.8601318372193966e-11,
  'PEP_Pi_pc': -0.27178338242599925}}

In [13]:
temp2 = temp.copy()
temp2.reactions.ATP_ADP_Pi_pc.lower_bound = -0.266793
temp2.reactions.ATP_ADP_Pi_pc.upper_bound = -0.266793

sol = flux_analysis.parsimonious.optimize_minimal_flux(temp2)

total = JATPase
for rxn in temp2.metabolites.ATP_p.reactions:
    if round(rxn.flux,3) != 0:
        coeff1 = rxn.metabolites[temp2.metabolites.ATP_p]
        coeff2 = rxn.metabolites[temp2.metabolites.aATP_p]
        ATPflux = sol.fluxes[rxn.id]*(coeff1+coeff2)
        print(rxn.id+"\t"+str(ATPflux)+"="+str(total))
        if rxn.id == "ATP_ADP_Pi_pc":
            total = total + ATPflux
            print(ATPflux)
print("Extra APTase flux ="+str(total))

ADENYL_KIN_RXN_p	-0.024086189956044567=0
ATP_ADP_Pi_pc	0.266793=0
0.266793
ASPARTATEKIN_RXN_p	-0.10806237580343216=0.266793
GDPKIN_RXN_p	-0.004014364992674087=0.266793
CARBAMATE_KINASE_RXN_p	-0.0040143649926740865=0.266793
ARGSUCCINSYN_RXN_p	-0.004014364992674086=0.266793
PEPDEPHOS_RXN_p	4.99142889839082=0.266793
ACETYLGLUTKIN_RXN_p	-0.0040143649926740865=0.266793
HOMOSERKIN_RXN_p	-0.08679708885251895=0.266793
SHIKIMATE_KINASE_RXN_p	-0.06271089601996639=0.266793
SULFATE_ADENYLYLTRANS_RXN_p	-0.008028729985346426=0.266793
ATPPHOSPHORIBOSYLTRANS_RXN_p	-0.0040143649926740865=0.266793
PRPPSYN_RXN_p	-0.008028729985348173=0.266793
GLUTAMINESYN_RXN_p	-4.940436062824799=0.266793
Extra APTase flux =0.266793


In [16]:
met = temp2.metabolites.PYRUVATE_p
for rxn in met.reactions:
    if round(rxn.flux,3) != 0:
        coeff1 = rxn.metabolites[met]
        coeff2 = 0#rxn.metabolites[temp2.metabolites.aATP_p]
        flux = sol.fluxes[rxn.id]*1#(coeff1+coeff2)
        print(rxn.id+"\t"+str(flux))

PYRUVDEH_RXN_p	0.02376070232788035
CYSTATHIONINE_BETA_LYASE_RXN_p	0.004014364992672349
PYRUVATE_pc	4.845067307986973
ACETOLACTSYN_RXN_p	0.04632794554459786
ACETOOHBUTSYN_RXN_p	0.01670844002120415
ANTHRANSYN_RXN_p	0.004014364992674087
DIHYDRODIPICSYN_RXN_p	0.021265286950913204
PEPDEPHOS_RXN_p	4.99142889839082


In [15]:
met = temp2.metabolites.PHOSPHO_ENOL_PYRUVATE_p
for rxn in met.reactions:
    if round(rxn.flux,3) != 0:
        coeff1 = rxn.metabolites[met]
        coeff2 = 0#rxn.metabolites[temp2.metabolites.aATP_p]
        flux = sol.fluxes[rxn.id]*(coeff1+coeff2)
        print(rxn.id+"\t"+str(flux))

DAHPSYN_RXN_p	-0.0627108960199664
2_PERIOD_5_PERIOD_1_PERIOD_19_RXN_p	-0.06271089601996639
PEPDEPHOS_RXN_p	-4.99142889839082
PEP_Pi_pc	5.116850690430752
